<a href="https://colab.research.google.com/github/Mathavk1606/Real-Estate-Demand-Prediction/blob/main/Real_Estate_Demand_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real Estate Demand Prediction**

## **📋 Table of Contents**

Introduction & Problem Statement

Dataset Overview

Exploratory Data Analysis

Feature Engineering

Model Development

Results & Insights

Conclusions

## **🎯 Introduction & Problem Statement**
In China’s fast-evolving and highly dynamic housing market, accurately forecasting residential demand is vital for investment and development decisions. This competition challenges you to develop a machine learning model that predicts each sector's monthly sales for newly launched private residential projects, using historical transaction data, market conditions, and other relevant features.

# Real Estate and City Data Dictionary

## Table of Contents
1. [Pre-owned House Transactions](#pre-owned-house-transactions)
2. [Pre-owned House Transactions (Nearby Sectors)](#pre-owned-house-transactions-nearby-sectors)
3. [Land Transactions](#land-transactions)
4. [Land Transactions (Nearby Sectors)](#land-transactions-nearby-sectors)
5. [New House Transactions](#new-house-transactions)
6. [New House Transactions (Nearby Sectors)](#new-house-transactions-nearby-sectors)
7. [Sector POI (Points of Interest)](#sector-poi-points-of-interest)
8. [City Search Index](#city-search-index)
9. [City Indexes](#city-indexes)

---

## Pre-owned House Transactions
**File:** `train/pre_owned_house_transactions.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector where the transaction occurred | Text |
| `area_pre_owned_house_transactions` | The total area of pre-owned house transactions | Square meters (m²) |
| `amount_pre_owned_house_transactions` | The total monetary value of pre-owned house transactions | 10,000 yuan |
| `num_pre_owned_house_transactions` | The total number of pre-owned house transactions | Count |
| `price_pre_owned_house_transactions` | The average price per square meter of pre-owned house transactions | Yuan per m² |

---

## Pre-owned House Transactions (Nearby Sectors)
**File:** `train/pre_owned_house_transactions_nearby_sectors.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector of interest | Text |
| `area_pre_owned_house_transactions_nearby_sectors` | The total area of pre-owned house transactions in nearby sectors | Square meters (m²) |
| `amount_pre_owned_house_transactions_nearby_sectors` | The total monetary value of pre-owned house transactions in nearby sectors | 10,000 yuan |
| `num_pre_owned_house_transactions_nearby_sectors` | The total number of pre-owned house transactions in nearby sectors | Count |
| `price_pre_owned_house_transactions_nearby_sectors` | The average price per square meter of pre-owned house transactions in nearby sectors | Yuan per m² |

---

## Land Transactions
**File:** `train/land_transactions.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector where the land transaction occurred | Text |
| `num_land_transactions` | The total number of land transactions | Count |
| `construction_area` | The total area of land designated for construction | Square meters (m²) |
| `planned_building_area` | The total planned building area on the transacted land | Square meters (m²) |
| `transaction_amount` | The total monetary value of land transactions | 10,000 yuan |

---

## Land Transactions (Nearby Sectors)
**File:** `train/land_transactions_nearby_sectors.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector of interest | Text |
| `num_land_transactions_nearby_sectors` | The total number of land transactions in nearby sectors | Count |
| `construction_area_nearby_sectors` | The total area of land designated for construction in nearby sectors | Square meters (m²) |
| `planned_building_area_nearby_sectors` | The total planned building area on transacted land in nearby sectors | Square meters (m²) |
| `transaction_amount_nearby_sectors` | The total monetary value of land transactions in nearby sectors | 10,000 yuan |

---

## New House Transactions
**File:** `train/new_house_transactions.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector where the new house transaction occurred | Text |
| `num_new_house_transactions` | The total number of new house transactions | Count |
| `area_new_house_transactions` | The total area of new house transactions | Square meters (m²) |
| `price_new_house_transactions` | The average price per square meter of new house transactions | Yuan per m² |
| `amount_new_house_transactions` | The total monetary value of new house transactions | 10,000 yuan |
| `area_per_unit_new_house_transactions` | The average area per new house transaction unit | m² per unit |
| `total_price_per_unit_new_house_transactions` | The average total price per new house transaction unit | 10,000 yuan per unit |
| `num_new_house_available_for_sale` | The total number of new houses available for sale | Count |
| `area_new_house_available_for_sale` | The total area of new houses available for sale | Square meters (m²) |
| `period_new_house_sell_through` | The estimated time to sell all available new houses | Months |

---

## New House Transactions (Nearby Sectors)
**File:** `train/new_house_transactions_nearby_sectors.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month of the transaction | Date (YYYY-MM) |
| `sector` | The specific geographic sector of interest | Text |
| `num_new_house_transactions_nearby_sectors` | The total number of new house transactions in nearby sectors | Count |
| `area_new_house_transactions_nearby_sectors` | The total area of new house transactions in nearby sectors | Square meters (m²) |
| `price_new_house_transactions_nearby_sectors` | The average price per square meter of new house transactions in nearby sectors | Yuan per m² |
| `amount_new_house_transactions_nearby_sectors` | The total monetary value of new house transactions in nearby sectors | 10,000 yuan |
| `area_per_unit_new_house_transactions_nearby_sectors` | The average area per new house transaction unit in nearby sectors | m² per unit |
| `total_price_per_unit_new_house_transactions_nearby_sectors` | The average total price per new house transaction unit in nearby sectors | 10,000 yuan per unit |
| `num_new_house_available_for_sale_nearby_sectors` | The total number of new houses available for sale in nearby sectors | Count |
| `area_new_house_available_for_sale_nearby_sectors` | The total area of new houses available for sale in nearby sectors | Square meters (m²) |
| `period_new_house_sell_through_nearby_sectors` | The estimated time to sell all available new houses in nearby sectors | Months |

---

## Sector POI (Points of Interest)
**File:** `train/sector_POI.csv`

### Basic Sector Information

| Field Name | Description | Unit |
|------------|-------------|------|
| `sector` | The specific geographic sector | Text |
| `sector_coverage` | The geographical extent or area covered by the sector | Area unit |
| `population_scale` | The general size of the population within the sector | Scale/Category |
| `residential_area` | The presence or extent of residential zones within the sector | Indicator |
| `office_building` | The presence or extent of office buildings within the sector | Indicator |
| `commercial_area` | The presence or extent of commercial zones within the sector | Indicator |

### Population Metrics

| Field Name | Description | Unit |
|------------|-------------|------|
| `resident_population` | The number of people residing in the sector | Count |
| `office_population` | The number of people working in offices within the sector | Count |

### Commercial and Retail

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_shops` | The total count of shops in the sector | Count |
| `catering` | The number or density of catering establishments | Count |
| `retail` | The number or density of retail establishments | Count |
| `hotel` | The number or density of hotel establishments | Count |
| `rentable_shops` | The number of shops available for rent | Count |
| `surrounding_housing_average_price` | The average price of housing in the surrounding area | Yuan |
| `surrounding_shop_average_rent` | The average rent of shops in the surrounding area | Yuan |

### Transportation

| Field Name | Description | Unit |
|------------|-------------|------|
| `transportation_station` | The number or density of transportation stations | Count |
| `bus_station_cnt` | The count of bus stations | Count |
| `subway_station_cnt` | The count of subway stations | Count |
| `transportation_facilities_service_bus_station` | The presence or density of bus stations | Indicator |
| `transportation_facilities_service_subway_station` | The presence or density of subway stations | Indicator |
| `transportation_facilities_service_airport_related` | The presence or density of airport-related facilities | Indicator |
| `transportation_facilities_service_port_terminal` | The presence or density of port or terminal facilities | Indicator |
| `transportation_facilities_service_train_station` | The presence or density of train stations | Indicator |
| `transportation_facilities_service_light_rail_station` | The presence or density of light rail stations | Indicator |
| `transportation_facilities_service_long_distance_bus_station` | The presence or density of long-distance bus stations | Indicator |

### Education Facilities

| Field Name | Description | Unit |
|------------|-------------|------|
| `education` | The number or density of educational facilities | Count |
| `education_training_school_education_middle_school` | The number or density of middle schools | Count |
| `education_training_school_education_primary_school` | The number or density of primary schools | Count |
| `education_training_school_education_kindergarten` | The number or density of kindergartens | Count |
| `education_training_school_education_research_institution` | The number or density of research institutions | Count |

### Leisure and Entertainment

| Field Name | Description | Unit |
|------------|-------------|------|
| `leisure_and_entertainment` | The number or density of leisure and entertainment venues | Count |
| `leisure_entertainment_entertainment_venue_game_arcade` | The number or density of game arcades | Count |
| `leisure_entertainment_entertainment_venue_party_house` | The number or density of party houses | Count |
| `leisure_entertainment_cultural_venue_cultural_palace` | The number or density of cultural palaces | Count |

### Medical and Health Facilities

| Field Name | Description | Unit |
|------------|-------------|------|
| `medical_health` | The number or density of general medical and health facilities | Count |
| `medical_health_specialty_hospital` | The number or density of specialty hospitals | Count |
| `medical_health_tcm_hospital` | The number or density of Traditional Chinese Medicine (TCM) hospitals | Count |
| `medical_health_physical_examination_institution` | The number or density of physical examination institutions | Count |
| `medical_health_veterinary_station` | The number or density of veterinary stations | Count |
| `medical_health_pharmaceutical_healthcare` | The number or density of pharmaceutical healthcare providers | Count |
| `medical_health_rehabilitation_institution` | The number or density of rehabilitation institutions | Count |
| `medical_health_first_aid_center` | The number or density of first aid centers | Count |
| `medical_health_blood_donation_station` | The number or density of blood donation stations | Count |
| `medical_health_disease_prevention_institution` | The number or density of disease prevention institutions | Count |
| `medical_health_general_hospital` | The number or density of general hospitals | Count |
| `medical_health_clinic` | The number or density of clinics | Count |

### Office and Industrial Buildings

| Field Name | Description | Unit |
|------------|-------------|------|
| `office_building_industrial_building_industrial_building` | The number or density of industrial buildings used as office spaces | Count |

### Store Categories

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_leisure_and_entertainment_stores` | The count of leisure and entertainment stores | Count |
| `number_of_other_stores` | The count of miscellaneous other stores | Count |
| `number_of_other_anchor_stores` | The count of other major or anchor stores | Count |
| `number_of_home_appliance_stores` | The count of home appliance stores | Count |
| `number_of_skincare_cosmetics_stores` | The count of skincare and cosmetics stores | Count |
| `number_of_fashion_stores` | The count of fashion stores | Count |
| `number_of_service_stores` | The count of service-oriented stores | Count |
| `number_of_jewelry_stores` | The count of jewelry stores | Count |
| `number_of_lifestyle_leisure_stores` | The count of lifestyle and leisure stores | Count |
| `number_of_supermarket_convenience_stores` | The count of supermarkets and convenience stores | Count |
| `number_of_catering_food_stores` | The count of catering and food stores | Count |

### Commercial Building Types

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_residential_commercial` | The count of commercial establishments within residential areas | Count |
| `number_of_office_building_commercial` | The count of commercial establishments within office buildings | Count |
| `number_of_commercial_buildings` | The count of dedicated commercial buildings | Count |
| `number_of_hypermarkets` | The count of hypermarkets | Count |
| `number_of_department_stores` | The count of department stores | Count |
| `number_of_shopping_centers` | The count of shopping centers | Count |
| `number_of_hotel_commercial` | The count of commercial establishments within hotels | Count |

### Shopping Mall Classifications

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_third_tier_shopping_malls_in_business_district` | The count of third-tier shopping malls within the business district | Count |
| `number_of_second_tier_shopping_malls_in_business_district` | The count of second-tier shopping malls within the business district | Count |
| `number_of_city_winner_malls` | The count of high-performing "city winner" malls | Count |
| `number_of_shopping_malls_with_street_facing_shops` | The count of shopping malls featuring street-facing shops | Count |
| `number_of_unranked_malls` | The count of shopping malls without a specific ranking | Count |
| `number_of_community_malls` | The count of community-focused malls | Count |
| `number_of_community_winner_malls` | The count of high-performing "community winner" malls | Count |
| `number_of_key_focus_malls` | The count of shopping malls identified for key focus | Count |

### Density Metrics

All density metrics represent the concentration or density of the corresponding feature within the sector coverage area.

#### Basic Density Indicators

| Field Name | Description | Unit |
|------------|-------------|------|
| `population_scale_dense` | The density of the population scale within the sector | Density measure |
| `residential_area_dense` | The density of residential areas within the sector | Density measure |
| `office_building_dense` | The density of office buildings within the sector | Density measure |
| `commercial_area_dense` | The density of commercial areas within the sector | Density measure |
| `resident_population_dense` | The density of the resident population within the sector | Density measure |
| `office_population_dense` | The density of the office population within the sector | Density measure |
| `number_of_shops_dense` | The density of shops within the sector | Density measure |
| `catering_dense` | The density of catering establishments within the sector | Density measure |
| `retail_dense` | The density of retail establishments within the sector | Density measure |
| `hotel_dense` | The density of hotel establishments within the sector | Density measure |
| `transportation_station_dense` | The density of transportation stations within the sector | Density measure |
| `education_dense` | The density of educational facilities within the sector | Density measure |
| `leisure_and_entertainment_dense` | The density of leisure and entertainment venues within the sector | Density measure |
| `bus_station_cnt_dense` | The density of bus stations | Density measure |
| `subway_station_cnt_dense` | The density of subway stations | Density measure |
| `rentable_shops_dense` | The density of rentable shops | Density measure |

#### Store Density Indicators

| Field Name | Description | Unit |
|------------|-------------|------|
| `leisure_entertainment_stores_dense` | The density of leisure and entertainment stores | Density measure |
| `other_stores_dense` | The density of miscellaneous other stores | Density measure |
| `other_anchor_stores_dense` | The density of other major or anchor stores | Density measure |
| `home_appliance_stores_dense` | The density of home appliance stores | Density measure |
| `skincare_cosmetics_stores_dense` | The density of skincare and cosmetics stores | Density measure |
| `fashion_stores_dense` | The density of fashion stores | Density measure |
| `service_stores_dense` | The density of service-oriented stores | Density measure |
| `jewelry_stores_dense` | The density of jewelry stores | Density measure |
| `lifestyle_leisure_stores_dense` | The density of lifestyle and leisure stores | Density measure |
| `supermarket_convenience_stores_dense` | The density of supermarkets and convenience stores | Density measure |
| `catering_food_stores_dense` | The density of catering and food stores | Density measure |

#### Commercial Building Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `residential_commercial_dense` | The density of commercial establishments within residential areas | Density measure |
| `office_building_commercial_dense` | The density of commercial establishments within office buildings | Density measure |
| `commercial_buildings_dense` | The density of dedicated commercial buildings | Density measure |
| `hypermarkets_dense` | The density of hypermarkets | Density measure |
| `department_stores_dense` | The density of department stores | Density measure |
| `shopping_centers_dense` | The density of shopping centers | Density measure |
| `hotel_commercial_dense` | The density of commercial establishments within hotels | Density measure |

#### Mall Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `third_tier_shopping_malls_in_business_district_dense` | The density of third-tier shopping malls within the business district | Density measure |
| `second_tier_shopping_malls_in_business_district_dense` | The density of second-tier shopping malls within the business district | Density measure |
| `city_winner_malls_dense` | The density of high-performing "city winner" malls | Density measure |
| `shopping_malls_with_street_facing_shops_dense` | The density of shopping malls featuring street-facing shops | Density measure |
| `unranked_malls_dense` | The density of shopping malls without a specific ranking | Density measure |
| `community_malls_dense` | The density of community-focused malls | Density measure |
| `community_winner_malls_dense` | The density of high-performing "community winner" malls | Density measure |
| `key_focus_malls_dense` | The density of shopping malls identified for key focus | Density measure |

#### Transportation Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `transportation_facilities_service_bus_station_dense` | The density of bus stations | Density measure |
| `transportation_facilities_service_subway_station_dense` | The density of subway stations | Density measure |
| `transportation_facilities_service_airport_related_dense` | The density of airport-related facilities | Density measure |
| `transportation_facilities_service_port_terminal_dense` | The density of port or terminal facilities | Density measure |
| `transportation_facilities_service_train_station_dense` | The density of train stations | Density measure |
| `transportation_facilities_service_light_rail_station_dense` | The density of light rail stations | Density measure |
| `transportation_facilities_service_long_distance_bus_station_dense` | The density of long-distance bus stations | Density measure |

#### Entertainment and Culture Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `leisure_entertainment_entertainment_venue_game_arcade_dense` | The density of game arcades | Density measure |
| `leisure_entertainment_entertainment_venue_party_house_dense` | The density of party houses | Density measure |
| `leisure_entertainment_cultural_venue_cultural_palace_dense` | The density of cultural palaces | Density measure |

#### Industrial and Office Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `office_building_industrial_building_industrial_building_dense` | The density of industrial buildings used as office spaces | Density measure |

#### Medical Facility Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `medical_health_dense` | The density of general medical and health facilities | Density measure |
| `medical_health_specialty_hospital_dense` | The density of specialty hospitals | Density measure |
| `medical_health_tcm_hospital_dense` | The density of Traditional Chinese Medicine (TCM) hospitals | Density measure |
| `medical_health_physical_examination_institution_dense` | The density of physical examination institutions | Density measure |
| `medical_health_veterinary_station_dense` | The density of veterinary stations | Density measure |
| `medical_health_pharmaceutical_healthcare_dense` | The density of pharmaceutical healthcare providers | Density measure |
| `medical_health_rehabilitation_institution_dense` | The density of rehabilitation institutions | Density measure |
| `medical_health_first_aid_center_dense` | The density of first aid centers | Density measure |
| `medical_health_blood_donation_station_dense` | The density of blood donation stations | Density measure |
| `medical_health_disease_prevention_institution_dense` | The density of disease prevention institutions | Density measure |
| `medical_health_general_hospital_dense` | The density of general hospitals | Density measure |
| `medical_health_clinic_dense` | The density of clinics | Density measure |

#### Education Facility Density

| Field Name | Description | Unit |
|------------|-------------|------|
| `education_training_school_education_middle_school_dense` | The density of middle schools | Density measure |
| `education_training_school_education_primary_school_dense` | The density of primary schools | Density measure |
| `education_training_school_education_kindergarten_dense` | The density of kindergartens | Density measure |
| `education_training_school_education_research_institution_dense` | The density of research institutions | Density measure |

---

## City Search Index
**File:** `train/city_search_index.csv`

| Field Name | Description | Unit |
|------------|-------------|------|
| `month` | The month the search data was recorded | Date (YYYY-MM) |
| `keyword` | The specific search term | Text |
| `source` | The origin or platform of the search data | Text |
| `search_volume` | The total number of searches for the keyword | Count |

---

## City Indexes
**File:** `train/city_indexes.csv`

### Population Demographics

| Field Name | Description | Unit |
|------------|-------------|------|
| `city_indicator_data_year` | The year to which the city indicator data pertains | Year (YYYY) |
| `year_end_registered_population_10k` | The registered population at year-end | 10,000 persons |
| `total_households_10k` | The total number of households | 10,000 households |
| `year_end_resident_population_10k` | The permanent resident population at year-end | 10,000 persons |
| `national_year_end_total_population_10k` | The national total population at year-end | 10,000 persons |
| `resident_registered_ratio` | The ratio of permanent residents to registered population | Ratio |
| `under_18_10k` | The population under 18 years old | 10,000 persons |
| `18_60_years_10k` | The population aged 18 to 60 years old | 10,000 persons |
| `over_60_years_10k` | The population over 60 years old | 10,000 persons |
| `total` | The total population count | Count |
| `under_18_percent` | The percentage of the population under 18 years old | Percentage |
| `18_60_years_percent` | The percentage of the population aged 18 to 60 years old | Percentage |
| `over_60_years_percent` | The percentage of the population over 60 years old | Percentage |
| `national_population_share` | The city's share of the national population | Percentage/Ratio |

### Employment

| Field Name | Description | Unit |
|------------|-------------|------|
| `year_end_total_employed_population_10k` | The total employed population at year-end | 10,000 persons |
| `year_end_urban_non_private_employees_10k` | The number of urban non-private unit employees at year-end | 10,000 persons |
| `private_individual_and_other_employees_10k` | The number of private, individual, and other employees | 10,000 persons |
| `private_individual_ratio` | The proportion of private and individual employees | Ratio |
| `employed_population` | The total number of employed individuals | Count |
| `primary_industry_percent` | The percentage of the employed population in the primary industry | Percentage |
| `secondary_industry_percent` | The percentage of the employed population in the secondary industry | Percentage |
| `tertiary_industry_percent` | The percentage of the employed population in the tertiary industry | Percentage |
| `white_collar_service_vs_blue_collar_manufacturing_ratio` | The ratio of white-collar (service industry) to blue-collar (manufacturing industry) population | Ratio |

### Economic Indicators

| Field Name | Description | Unit |
|------------|-------------|------|
| `gdp_100m` | The Gross Domestic Product (GDP) | 100 million yuan |
| `primary_industry_100m` | The output value of the primary industry | 100 million yuan |
| `secondary_industry_100m` | The output value of the secondary industry | 100 million yuan |
| `tertiary_industry_100m` | The output value of the tertiary industry | 100 million yuan |
| `gdp_per_capita_yuan` | The GDP per capita | Yuan |
| `national_gdp_100m` | The national GDP | 100 million yuan |
| `national_economic_primacy` | An indicator of the city's economic dominance compared to the nation | Indicator |
| `gdp_population_ratio` | The ratio of the city's GDP primacy to its national population share | Ratio |
| `secondary_industry_development_gdp_share` | The share of the secondary industry in the GDP, indicating its development | Percentage |
| `tertiary_industry_development_gdp_share` | The share of the tertiary industry in the GDP, indicating its development | Percentage |

### Government Finance

| Field Name | Description | Unit |
|------------|-------------|------|
| `general_public_budget_revenue_100m` | The general public budget revenue | 100 million yuan |
| `personal_income_tax_100m` | The personal income tax collected | 100 million yuan |
| `per_capita_personal_income_tax_yuan` | The per capita personal income tax | Yuan |
| `general_public_budget_expenditure_100m` | The general public budget expenditure | 100 million yuan |
| `science_expenditure_10k` | The expenditure on science | 10,000 yuan |
| `education_expenditure_10k` | The expenditure on education | 10,000 yuan |

### Consumer Metrics

| Field Name | Description | Unit |
|------------|-------------|------|
| `total_retail_sales_of_consumer_goods_100m` | The total retail sales of consumer goods | 100 million yuan |
| `retail_sales_growth_rate` | The growth rate of retail sales | Percentage |
| `urban_consumer_price_index_previous_year_100` | The urban consumer price index, with the previous year set as 100 | Index (base=100) |
| `engel_coefficient` | The Engel coefficient, indicating the proportion of income spent on food | Coefficient |

### Income and Wages

| Field Name | Description | Unit |
|------------|-------------|------|
| `annual_average_wage_urban_non_private_employees_yuan` | The annual average wage of urban non-private unit employees | Yuan |
| `annual_average_wage_urban_non_private_on_duty_employees_yuan` | The annual average wage of urban non-private on-duty employees | Yuan |
| `per_capita_disposable_income_absolute_yuan` | The absolute value of per capita disposable income | Yuan |
| `per_capita_disposable_income_index_previous_year_100` | The per capita disposable income index, with the previous year set as 100 | Index (base=100) |

### Housing

| Field Name | Description | Unit |
|------------|-------------|------|
| `per_capita_housing_area_sqm` | The per capita housing area | Square meters (m²) |

### Education Infrastructure

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_universities` | The total count of universities and colleges | Count |
| `university_students_10k` | The number of university students | 10,000 students |
| `number_of_middle_schools` | The total count of middle schools | Count |
| `middle_school_students_10k` | The number of middle school students | 10,000 students |
| `number_of_primary_schools` | The total count of primary schools | Count |
| `primary_school_students_10k` | The number of primary school students | 10,000 students |
| `number_of_kindergartens` | The total count of kindergartens | Count |
| `kindergarten_students_10k` | The number of kindergarten students | 10,000 students |

### Healthcare Infrastructure

| Field Name | Description | Unit |
|------------|-------------|------|
| `hospitals_health_centers` | The total count of hospitals and health centers | Count |
| `hospital_beds_10k` | The number of hospital beds | 10,000 beds |
| `health_technical_personnel_10k` | The number of health technical personnel | 10,000 persons |
| `doctors_10k` | The number of doctors | 10,000 doctors |

### Transportation Infrastructure

| Field Name | Description | Unit |
|------------|-------------|------|
| `road_length_km` | The total length of roads | Kilometers (km) |
| `road_area_10k_sqm` | The total area of roads | 10,000 square meters |
| `per_capita_urban_road_area_sqm` | The per capita urban road area | Square meters (m²) |
| `number_of_operating_bus_lines` | The total count of operating bus lines | Count |
| `operating_bus_line_length_km` | The total length of operating bus lines | Kilometers (km) |

### Internet Connectivity

| Field Name | Description | Unit |
|------------|-------------|------|
| `internet_broadband_access_subscribers_10k` | The number of internet broadband access subscribers | 10,000 subscribers |
| `internet_broadband_access_ratio` | The ratio of internet broadband access | Ratio/Percentage |

### Industrial and Investment

| Field Name | Description | Unit |
|------------|-------------|------|
| `number_of_industrial_enterprises_above_designated_size` | The total count of industrial enterprises above a designated size | Count |
| `total_current_assets_10k` | The total current assets | 10,000 yuan |
| `total_fixed_assets_10k` | The total fixed assets | 10,000 yuan |
| `main_business_taxes_and_surcharges_10k` | The main business taxes and surcharges | 10,000 yuan |
| `total_fixed_asset_investment_10k` | The total fixed asset investment | 10,000 yuan |
| `real_estate_development_investment_completed_10k` | The completed real estate development investment | 10,000 yuan |
| `residential_development_investment_completed_10k` | The completed residential development investment | 10,000 yuan |

---

## Notes and Conventions

### Monetary Units
- **10,000 yuan**: Values are expressed in units of ten thousand Chinese yuan (万元)
- **100 million yuan**: Values are expressed in units of one hundred million Chinese yuan (亿元)
- **Yuan**: Standard Chinese currency unit (¥)

### Population Units
- **10,000 persons**: Population counts expressed in units of ten thousand people (万人)

### Area Units
- **Square meters (m²)**: Standard metric unit for area measurements
- **10,000 square meters**: Larger areas expressed in units of ten thousand square meters

### Density Measures
Density metrics in the Sector POI dataset represent normalized values that account for the sector coverage area. These allow for fair comparison across sectors of different sizes.

### Date Formats
- **Month**: Typically formatted as YYYY-MM (e.g., 2023-01)
- **Year**: Four-digit year (e.g., 2023)

### Data Relationships
- **Primary datasets** contain transaction or indicator data for specific sectors
- **Nearby sectors datasets** aggregate data from geographically adjacent sectors to capture spillover effects and regional trends
- **POI (Points of Interest)** data provides detailed infrastructure and amenity information at the sector level
- **City-level indexes** provide macro-economic and demographic context

### Key Concepts

**Sector**: A geographic subdivision used for spatial analysis of real estate and urban characteristics.

**Pre-owned Houses**: Previously occupied residential properties being resold (also known as second-hand or resale housing).

**New Houses**: Newly constructed residential properties being sold for the first time.

**Land Transactions**: Sales of land parcels, typically for future development.

**Nearby Sectors**: Adjacent or surrounding geographic sectors used to capture regional market dynamics.

**POI (Points of Interest)**: Locations of various facilities, services, and amenities within a sector.

**Density Metrics**: Normalized measures that account for the geographic size of sectors, enabling fair comparisons.

---

## Data Quality Considerations

When working with these datasets:

1. **Missing Values**: Some sectors may have missing data for certain time periods or metrics
2. **Temporal Coverage**: Different datasets may have different time ranges
3. **Aggregation Levels**: Pay attention to whether data is aggregated (totals) or averaged
4. **Unit Consistency**: Always verify the unit of measurement when performing calculations
5. **Seasonal Patterns**: Real estate transactions often exhibit seasonal variations
6. **Regional Variations**: Different sectors may have vastly different characteristics based on urban vs suburban locations

---

## Common Analysis Use Cases

This data dictionary supports various analytical tasks:

- **Price Prediction**: Forecasting real estate prices using historical transactions and POI features
- **Market Segmentation**: Identifying distinct real estate market clusters based on sector characteristics
- **Trend Analysis**: Tracking temporal patterns in transactions, prices, and inventory
- **Spatial Analysis**: Understanding how nearby sector characteristics influence local markets
- **Demand Forecasting**: Predicting future transaction volumes and inventory needs
- **Infrastructure Planning**: Analyzing the relationship between amenities and real estate activity
- **Economic Impact Assessment**: Evaluating how city-level economic indicators affect local markets

---

In [9]:
# Core Libraries
import numpy as np
import pandas as pd
import warnings
import os
import polars as pl
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
COLORS = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']

# Statistical Analysis
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr

print("✅ All libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Polars version: {pl.__version__}")

✅ All libraries imported successfully!
📦 Pandas version: 2.2.2
📦 NumPy version: 2.0.2
📦 Polars version: 1.25.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

currPath = os.getcwd()
#print(f"Current directory before changing: {currPath}")

os.chdir("/content/drive/MyDrive/china-real-estate-demand-prediction/train")
#print(f"Current directory after changing: {os.getcwd()}")

dir_list = os.listdir(os.getcwd())
#print("Files in the current directory:")
print(dir_list)
print(len(dir_list))

['land_transactions.csv', 'new_house_transactions.csv', 'sector_POI.csv', 'pre_owned_house_transactions.csv', 'new_house_transactions_nearby_sectors.csv', 'land_transactions_nearby_sectors.csv', 'city_search_index.csv', 'pre_owned_house_transactions_nearby_sectors.csv', 'city_indexes.csv']
9


In [4]:
print("✓ Pre requisites added")
month_codes = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}

✓ Pre requisites added


In [10]:
for i in dir_list:
    df = pd.read_csv(os.path.join(os.getcwd(), i))
    print(f"\n{'='*50}")
    print(f"File: {i}")
    print(f"{'='*50}")

    # if 'month' in df.columns:
    #     # Convert 'month' to datetime objects
    #     df['month'] = pd.to_datetime(df['month'])

    #     # Extract year and month abbreviation
    #     year_values = df['month'].dt.year.astype(str)  # Convert year to string
    #     month_values = df['month'].dt.month.map(month_codes)

    #     # Create the 'id' column
    #     df.insert(0, 'id', year_values + ' ' + month_values)

    #     df.drop('month', axis=1, inplace=True)


    display(df.head(10))
    print(f"\nShape: {df.shape}")

print("✓ Pre requisites added") # This line seems out of place here, it was probably intended for the setup cells.


File: land_transactions.csv


,month,sector,num_land_transactions,construction_area,planned_building_area,transaction_amount
0,2019-Jan,sector 74,0,0.0,0.0,0.0
1,2019-Jan,sector 35,0,0.0,0.0,0.0
2,2019-Jan,sector 23,0,0.0,0.0,0.0
3,2019-Jan,sector 80,0,0.0,0.0,0.0
4,2019-Jan,sector 53,0,0.0,0.0,0.0
5,2019-Jan,sector 84,0,0.0,0.0,0.0
6,2019-Jan,sector 16,0,0.0,0.0,0.0
7,2019-Jan,sector 27,0,0.0,0.0,0.0
8,2019-Jan,sector 82,0,0.0,0.0,0.0
9,2019-Jan,sector 14,0,0.0,0.0,0.0



Shape: (5896, 6)

File: new_house_transactions.csv


,month,sector,num_new_house_transactions,area_new_house_transactions,price_new_house_transactions,amount_new_house_transactions,area_per_unit_new_house_transactions,total_price_per_unit_new_house_transactions,num_new_house_available_for_sale,area_new_house_available_for_sale,period_new_house_sell_through
0,2019-Jan,sector 1,52,4906,28184,13827.14,94,265.91,159.0,15904.0,3.78
1,2019-Jan,sector 2,145,15933,17747,28277.73,110,195.02,1491.0,175113.0,12.29
2,2019-Jan,sector 4,6,725,28004,1424.21,127,356.05,40.0,6826.0,5.95
3,2019-Jan,sector 5,2,212,37432,792.10,106,396.05,161.0,17173.0,83.95
4,2019-Jan,sector 6,5,773,15992,607.94,95,151.99,189.0,19696.0,14.27
5,2019-Jan,sector 7,93,12678,31018,39326.10,136,422.86,867.0,124626.0,7.74
6,2019-Jan,sector 8,51,7183,14555,10454.68,141,204.99,1719.0,261360.0,40.71
7,2019-Jan,sector 9,13,840,52694,4170.55,66,347.55,95.0,14612.0,8.36
8,2019-Jan,sector 10,40,3629,30431,11043.29,91,276.08,209.0,19380.0,1.73
9,2019-Jan,sector 11,80,8935,11871,10606.75,112,132.58,602.0,118890.0,16.06



Shape: (5433, 11)

File: sector_POI.csv


...


Shape: (4020, 4)

File: pre_owned_house_transactions_nearby_sectors.csv


,month,sector,num_pre_owned_house_transactions_nearby_sectors,area_pre_owned_house_transactions_nearby_sectors,amount_pre_owned_house_transactions_nearby_sectors,price_pre_owned_house_transactions_nearby_sectors
0,2019-Jan,sector 1,6.750000,733.000000,1247.038000,17012.79673
1,2019-Jan,sector 2,64.181818,5339.000000,20880.242820,39108.90208
2,2019-Jan,sector 3,77.714286,7457.142857,17376.214860,23301.43755
3,2019-Jan,sector 4,57.666667,5109.666667,19021.152670,37225.81904
4,2019-Jan,sector 5,45.428571,3763.500000,15800.741430,41984.16747
5,2019-Jan,sector 6,38.300000,3201.500000,10070.514000,31455.61143
6,2019-Jan,sector 7,41.428571,3914.714286,9200.657143,23502.75517
7,2019-Jan,sector 8,57.500000,4886.900000,20190.862500,41316.29970
8,2019-Jan,sector 9,53.222222,4677.000000,19835.218440,42410.13138
9,2019-Jan,sector 10,55.777778,5630.444444,9644.433333,17129.08000



Shape: (5427, 6)

File: city_indexes.csv


,city_indicator_data_year,year_end_registered_population_10k,total_households_10k,year_end_resident_population_10k,year_end_total_employed_population_10k,year_end_urban_non_private_employees_10k,private_individual_and_other_employees_10k,private_individual_ratio,national_year_end_total_population_10k,resident_registered_ratio,...,internet_broadband_access_ratio,number_of_industrial_enterprises_above_designated_size,total_current_assets_10k,total_fixed_assets_10k,main_business_taxes_and_surcharges_10k,total_fixed_asset_investment_10k,real_estate_development_investment_completed_10k,residential_development_investment_completed_10k,science_expenditure_10k,education_expenditure_10k
0,2017,897.87,295.0211,1449.84,862.33,329.17,708.0004,0.821032,139008,1.614755,...,0.580841,4664,96779199,39013240.0,4012517.0,NaN,27028935.0,17694861.0,1712569.0,4043335.0
1,2018,927.69,305.9851,1490.44,1102.36,348.65,753.7146,0.683728,139538,1.606614,...,0.586586,4675,99067099,NaN,4297060.0,NaN,27019323.0,17466618.0,1636655.0,4408209.0
2,2019,953.72,313.8455,1530.59,1125.89,400.22,725.6720,0.644532,140005,1.604863,...,0.593287,5804,110643480,39934058.0,4862051.0,NaN,31022573.0,20870742.0,2439456.0,5239743.0
3,2020,985.11,322.1068,1874.03,1158.01,419.36,738.6462,0.637858,141212,1.902356,...,0.629148,6208,123664111,NaN,NaN,NaN,NaN,NaN,2241321.0,5585916.0
4,2021,1011.53,329.7224,1881.06,1163.44,426.94,736.5017,0.633038,141260,1.859619,...,0.686762,6757,142064810,NaN,NaN,NaN,NaN,NaN,2012478.0,5890857.0
5,2022,1034.91,336.1935,1873.41,1119.82,424.90,694.9159,0.620560,141175,1.810215,...,0.736798,6878,159482630,NaN,NaN,NaN,NaN,NaN,1982136.0,6269391.0
6,2022,1034.91,336.1935,1873.41,1119.82,424.90,694.9159,0.620560,141175,1.810215,...,0.736798,6878,159482630,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Shape: (7, 74)
✓ Pre requisites added


In [6]:
for i in dir_list:
    df = pd.read_csv(os.path.join(os.getcwd(), i))
    print(f"\n{'='*60}")
    print(f"📁 File: {i}")
    print(f"{'='*60}")
    print(f"Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")
    print(f"Total null values: {df.isnull().sum().sum():,}")
    print(f"Duplicated rows: {df.duplicated().sum():,}")
    print(f"\nData Types:")
    for dtype, count in df.dtypes.value_counts().items():
        print(f"  {dtype}: {count} columns")

    null_cols = df.isnull().sum()
    if null_cols.sum() > 0:
        print(f"\nColumns with null values:")
        for col, count in null_cols[null_cols > 0].items():
            print(f"  {col}: {count:,} ({count/len(df)*100:.1f}%)")


📁 File: land_transactions.csv
Rows: 5,896 | Columns: 6
Total null values: 0
Duplicated rows: 0

Data Types:
  float64: 3 columns
  object: 2 columns
  int64: 1 columns

📁 File: new_house_transactions.csv
Rows: 5,433 | Columns: 11
Total null values: 42
Duplicated rows: 0

Data Types:
  float64: 5 columns
  int64: 4 columns
  object: 2 columns

Columns with null values:
  num_new_house_available_for_sale: 14 (0.3%)
  area_new_house_available_for_sale: 14 (0.3%)
  period_new_house_sell_through: 14 (0.3%)

📁 File: sector_POI.csv
Rows: 86 | Columns: 142
Total null values: 7
Duplicated rows: 0

Data Types:
  int64: 73 columns
  float64: 68 columns
  object: 1 columns

Columns with null values:
  surrounding_housing_average_price: 4 (4.7%)
  surrounding_shop_average_rent: 3 (3.5%)

📁 File: pre_owned_house_transactions.csv
Rows: 5,360 | Columns: 6
Total null values: 14
Duplicated rows: 0

Data Types:
  object: 2 columns
  int64: 2 columns
  float64: 2 columns

Columns with null values:
  pric

# **Group all tables into one for train.csv**

In [42]:
base_path = os.getcwd()

new_house_transactions = pd.read_csv(os.path.join(base_path, 'new_house_transactions.csv'))
new_house_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'new_house_transactions_nearby_sectors.csv'))
pre_owned_house_transactions = pd.read_csv(os.path.join(base_path, 'pre_owned_house_transactions.csv'))
pre_owned_house_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'pre_owned_house_transactions_nearby_sectors.csv'))
city_search_index = pd.read_csv(os.path.join(base_path, 'city_search_index.csv'))
sector_POI = pd.read_csv(os.path.join(base_path, 'sector_POI.csv'))
land_transactions = pd.read_csv(os.path.join(base_path, 'land_transactions.csv'))
land_transactions_nearby_sectors = pd.read_csv(os.path.join(base_path, 'land_transactions_nearby_sectors.csv'))
city_indexes = pd.read_csv(os.path.join(base_path, 'city_indexes.csv'))

print("✓ All datasets loaded successfully")

✓ All datasets loaded successfully


In [113]:
# Merge dataframes based on sectors
data1 = (
    pl.DataFrame(
        new_house_transactions['month'].unique()
    )
    .rename(
        {
            'column_0': 'month'
        }
    )
    .join(
        pl.DataFrame(list(new_house_transactions["sector"].unique()) + ["sector 95"])
        .rename({"column_0": "sector"}),
        how="cross",
    )
    .fill_null(-1)
    .join(
        pl.DataFrame(new_house_transactions),
        on=['month', 'sector'],
        how='left'
    )
    .fill_null(-1)
    .join(
        pl.DataFrame(new_house_transactions_nearby_sectors),
        on=['month', 'sector'],
        how='left'
    )
    .fill_null(-1)
    .join(
        pl.DataFrame(pre_owned_house_transactions),
        on=['month', 'sector'],
        how='left'
    )
    .join(
        pl.DataFrame(pre_owned_house_transactions_nearby_sectors),
        on=['month', 'sector'],
        how='left'
    )
    .fill_null(-1)
    .join(
        pl.DataFrame(land_transactions),
        on=['month', 'sector'],
        how='left'
    )
    .fill_null(-1)
    .join(
        pl.DataFrame(land_transactions_nearby_sectors),
        on=['month', 'sector'],
        how='left'
    )
    .fill_null(-1)
    .with_columns(
        pl.col('month').str.split('-').list.get(0).cast(pl.Int16).alias('year'),
    )
    .join(
        pl.DataFrame(city_indexes).rename({"city_indicator_data_year": "year"}),
        on=["year"],
        how="left"
    )
    .fill_null(-1)
    .join(pl.DataFrame(sector_POI), on=["sector"], how="left")
    .fill_null(-1)
    .with_columns(
        id = pl.col('month').str.split('-').list.get(0).cast(pl.Int16).cast(pl.Utf8) + ' ' + pl.col('month').str.split('-').list.get(1).cast(pl.Utf8) + '_' + pl.col('sector').cast(pl.Utf8)
    )
    .sort(['id'])
    .select(['id', pl.col('*').exclude('id')])
    .drop(['month','sector'])
)

#data1.null_count() - to check null counts

In [117]:
for col in data1.columns:
    if data1[col].dtype == pl.Int64 or data1[col].dtype == pl.Float64:
        c_min, c_max = data1[col].min(), data1[col].max()

        if c_min == 0 and c_max == 0:
            data1 = data1.drop(col)
            print(col, "0" * 20)
            continue

In [118]:
data1

id,num_new_house_transactions,area_new_house_transactions,price_new_house_transactions,amount_new_house_transactions,area_per_unit_new_house_transactions,total_price_per_unit_new_house_transactions,num_new_house_available_for_sale,area_new_house_available_for_sale,period_new_house_sell_through,num_new_house_transactions_nearby_sectors,area_new_house_transactions_nearby_sectors,price_new_house_transactions_nearby_sectors,amount_new_house_transactions_nearby_sectors,area_per_unit_new_house_transactions_nearby_sectors,total_price_per_unit_new_house_transactions_nearby_sectors,num_new_house_available_for_sale_nearby_sectors,area_new_house_available_for_sale_nearby_sectors,period_new_house_sell_through_nearby_sectors,area_pre_owned_house_transactions,amount_pre_owned_house_transactions,num_pre_owned_house_transactions,price_pre_owned_house_transactions,num_pre_owned_house_transactions_nearby_sectors,area_pre_owned_house_transactions_nearby_sectors,amount_pre_owned_house_transactions_nearby_sectors,price_pre_owned_house_transactions_nearby_sectors,num_land_transactions,construction_area,planned_building_area,transaction_amount,num_land_transactions_nearby_sectors,construction_area_nearby_sectors,planned_building_area_nearby_sectors,transaction_amount_nearby_sectors,year,year_end_registered_population_10k,…,shopping_centers_dense,hotel_commercial_dense,third_tier_shopping_malls_in_business_district_dense,second_tier_shopping_malls_in_business_district_dense,city_winner_malls_dense,shopping_malls_with_street_facing_shops_dense,unranked_malls_dense,community_malls_dense,community_winner_malls_dense,key_focus_malls_dense,transportation_facilities_service_bus_station_dense,transportation_facilities_service_subway_station_dense,transportation_facilities_service_airport_related_dense,transportation_facilities_service_port_terminal_dense,transportation_facilities_service_train_station_dense,transportation_facilities_service_light_rail_station_dense,transportation_facilities_service_long_distance_bus_station_dense,leisure_entertainment_entertainment_venue_game_arcade_dense,leisure_entertainment_entertainment_venue_party_house_dense,leisure_entertainment_cultural_venue_cultural_palace_dense,office_building_industrial_building_industrial_building_dense,medical_health_dense,medical_health_specialty_hospital_dense,medical_health_tcm_hospital_dense,medical_health_physical_examination_institution_dense,medical_health_veterinary_station_dense,medical_health_pharmaceutical_healthcare_dense,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense
str,i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,i16,f64,…,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2019 Apr_sector 1""",69,6935,38392,26626.68,101,385.89,141.0,12936.0,2.83,25.0,2611.777778,51527.35897,13457.80111,104.471111,538.312044,300.125,41819.875,16.21375,9228,39411.1,106,42708.17078,6.75,855.25,1718.0,20087.69366,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019,953.72,…,8.6000e-7,0.0,0.0,0.0,0.0,0,0.000005,0.0,0.0,0,0.000243,0.0000955,0.0,0.0,0.0,0,0.0,0.0000267,0.0000146,0.0000138,0,0.000563,0.0000318,0.0,0.0,0.0,0.000339,0.000113,0.0,0.0,8.6000e-7,0.0000409,0.0000378,0.0000155,0.0000284,0.0000632,0.0000138
"""2019 Apr_sector 10""",12,1181,28686,3387.26,98,282.27,123.0,11736.0,1.74,223.0,24602.0,23783.95049,58513.275,110.32287,262.391368,1648.0,199520.5,13.55,1636,3128.0,21,19119.8044,92.555556,9642.4

In [109]:
# Check for duplicate column names
duplicate_cols = pd.Series(data1.columns).duplicated().tolist()

print(f"\n5️⃣ DUPLICATE COLUMNS")
if not any(duplicate_cols):
    print(f"   ✅ No duplicate column names found")
else:
    print(f"   ⚠️ {sum(duplicate_cols)} duplicate column names detected:")
    print(f"   Duplicates: {pd.Series(data1.columns)[pd.Series(data1.columns).duplicated()].tolist()}")

# Check for duplicate rows
duplicates = data1.is_duplicated().sum()
print(f"\n4️⃣ DUPLICATE ROWS")
if duplicates == 0:
    print(f"   ✅ No duplicate records found")
else:
    print(f"   ⚠️ {duplicates} duplicate rows detected")


5️⃣ DUPLICATE COLUMNS
   ✅ No duplicate column names found

4️⃣ DUPLICATE ROWS
   ✅ No duplicate records found


In [120]:
data1.describe()

statistic,id,num_new_house_transactions,area_new_house_transactions,price_new_house_transactions,amount_new_house_transactions,area_per_unit_new_house_transactions,total_price_per_unit_new_house_transactions,num_new_house_available_for_sale,area_new_house_available_for_sale,period_new_house_sell_through,num_new_house_transactions_nearby_sectors,area_new_house_transactions_nearby_sectors,price_new_house_transactions_nearby_sectors,amount_new_house_transactions_nearby_sectors,area_per_unit_new_house_transactions_nearby_sectors,total_price_per_unit_new_house_transactions_nearby_sectors,num_new_house_available_for_sale_nearby_sectors,area_new_house_available_for_sale_nearby_sectors,period_new_house_sell_through_nearby_sectors,area_pre_owned_house_transactions,amount_pre_owned_house_transactions,num_pre_owned_house_transactions,price_pre_owned_house_transactions,num_pre_owned_house_transactions_nearby_sectors,area_pre_owned_house_transactions_nearby_sectors,amount_pre_owned_house_transactions_nearby_sectors,price_pre_owned_house_transactions_nearby_sectors,num_land_transactions,construction_area,planned_building_area,transaction_amount,num_land_transactions_nearby_sectors,construction_area_nearby_sectors,planned_building_area_nearby_sectors,transaction_amount_nearby_sectors,year,…,shopping_centers_dense,hotel_commercial_dense,third_tier_shopping_malls_in_business_district_dense,second_tier_shopping_malls_in_business_district_dense,city_winner_malls_dense,shopping_malls_with_street_facing_shops_dense,unranked_malls_dense,community_malls_dense,community_winner_malls_dense,key_focus_malls_dense,transportation_facilities_service_bus_station_dense,transportation_facilities_service_subway_station_dense,transportation_facilities_service_airport_related_dense,transportation_facilities_service_port_terminal_dense,transportation_facilities_service_train_station_dense,transportation_facilities_service_light_rail_station_dense,transportation_facilities_service_long_distance_bus_station_dense,leisure_entertainment_entertainment_venue_game_arcade_dense,leisure_entertainment_entertainment_venue_party_house_dense,leisure_entertainment_cultural_venue_cultural_palace_dense,office_building_industrial_building_industrial_building_dense,medical_health_dense,medical_health_specialty_hospital_dense,medical_health_tcm_hospital_dense,medical_health_physical_examination_institution_dense,medical_health_veterinary_station_dense,medical_health_pharmaceutical_healthcare_dense,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""7584""",7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,…,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0,7584.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,